In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

os.chdir("..")

from IPython.display import display

import sklearn.model_selection
import sklearn.metrics
import sklearn.datasets
from sklearn.ensemble import GradientBoostingClassifier

from Final_Data_Prep import remove_final_dummy, get_train_test, downsample

%matplotlib inline

import sys
sys.path.append("../")

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# read in fresh copy of df
df = pd.read_pickle('total_df.pckl.gz', compression = 'gzip')

In [5]:
'''
Data Prep (consistent with ADS as seen in Final_Gradient_Boosting_Model.ipynb)

Performing prep on total_df (here called df), not decoded_df as above in LIME
'''

#put earliest 85% of cases into training df, latest 15% of cases into test df
training, test = get_train_test(df, train_size=0.85, test_size=0.15)

# passing 50 to downsample function means training set will have 50% positive cases
training = downsample(training, 50)

X_train = training.drop('MHI', axis=1)
y_train = training['MHI']

X_test = test.drop('MHI', axis=1)
y_test = test['MHI']

In [6]:
clf_total = GradientBoostingClassifier(random_state=42, max_depth = 3, 
                                       min_samples_leaf = 10, learning_rate=.01, 
                                       n_estimators=1000)
clf_total.fit(X_train, y_train)

# Get predictions with original classifier
y_pred = clf_total.predict(X_test)

In [7]:
# set classifier list:
list_clf = [clf_total]

# get number of classifiers
num_clf = len(list_clf)

# make stacked array of predictions on test set
preds_test = np.array([y_pred])

# set X and y for full dataset
y_all = df['MHI'].copy()
X_all = df.drop(columns=['received_date', 'MHI'], inplace=False)

X_all_stack = np.array([X_all])

# initialize 2-d predictions arrays to 0
preds_all = np.zeros([num_clf, len(y_all)])

# get predictions on whole dataset
for i, clf in enumerate(list_clf):

    # get predictions on entire dataset
    preds_all[i] = clf.predict(X_all_stack[i])

ValueError: cannot copy sequence with size 289094 to array axis with dimension 5616

In [ ]:
def di_actual_and_pred(X, y_true, y_pred, col, priv_class):
    '''
    Calculate disparate impact in actual data and in classifier predictions
    DI = Pr(MHI==1|non-privileged class) / Pr(MHI==1| privileged class)
       = (# positive outcomes in the non-privileged class / # non-privileged)
           / (# positive outcomes in the privileged class / # privileged)

    X: features
    y_true: actual labels
    y_pred: 2-d array of labels predicted by different classifiers
    col: category (e.g. race, gender)
    priv_class: privileged class (e.g. white, male)
    '''
    
    # get number of classifiers
    num_clf = y_pred.shape[0]

    # set privileged column
    priv_col = col+'_'+priv_class

    # get number of privileged
    priv = X[X[priv_col]==1]
    num_priv = len(priv)

    # get number of privileged with actual positive
    ind_priv = priv.index
    y_priv = y_true.loc[ind_priv]
    true_num_pos_priv = y_priv.sum()

    # get number of non-privileged
    nonpriv = X[X[priv_col]==0]
    num_nonpriv = len(nonpriv)

    # get number of non-privileged with actual positive
    ind_nonpriv = nonpriv.index
    y_nonpriv = y_true.loc[ind_nonpriv]
    true_num_pos_nonpriv = y_nonpriv.sum()
    
    # get ilocs for actual positives in each class
    priv_ilocs = y_true.index.get_indexer_for((ind_priv))
    nonpriv_ilocs = y_true.index.get_indexer_for((ind_nonpriv))

    # get DI in original data
    DI_original = (true_num_pos_nonpriv / num_nonpriv) / (true_num_pos_priv / num_priv)
    print('Disparate impact on {} in original data: {}'.format(col, round(DI_original, 4)))
    
    # initialize DI_pred to array of 1s
    DI_pred = np.ones([num_clf])
    
    # calculate DI in predictions for each classifier
    for i in range(num_clf):
        
        # get number of privileged with positive prediction
        pred_num_pos_priv = y_pred[i][priv_ilocs].sum()

        # get number of non-privileged with positive prediction
        pred_num_pos_nonpriv = y_pred[i][nonpriv_ilocs].sum()

        # get DI in predictions
        DI_pred[i] = (pred_num_pos_nonpriv / num_nonpriv) / (pred_num_pos_priv / num_priv)
        print('Disparate impact on {} in classifier {} predictions: {}'.format(col, i+1, round(DI_pred[i], 4)))
    
    return DI_original, DI_pred

In [ ]:
# get DI measures

print('Test Set')
di_race_og_test, di_race_pred_test = di_actual_and_pred(X_test, y_test, preds_test, 'race', 'white')
di_gender_og_test, di_gender_pred_test = di_actual_and_pred(X_test, y_test, preds_test, 'gender', 'male')

print('\nEntire Dataset')
di_race_og_all, di_race_pred_all = di_actual_and_pred(X_all, y_all, preds_all, 'race', 'white')
di_gender_og_all, di_gender_pred_all = di_actual_and_pred(X_all, y_all, preds_all, 'gender', 'male')

In [48]:
# # add dummy data to actual from above
# # actual data will come from proxy-sensitive classifiers

# di_race_og_test = 0.5719
# di_race_pred_test = 0.5192
# di_race_og_all = 0.4569
# di_race_pred_all = 0.5289

# di_gender_og_test = 2.4247
# di_gender_pred_test = 3.2531
# di_gender_og_all = 3.5604
# di_gender_pred_all = 3.1462

In [ ]:
# put DI measures into dataframe


multi_index = pd.MultiIndex.from_product([['Test Set', 'All Data'],
                                          ['Actual',
                                           'Predictions']])

race_list = [di_race_og_test, di_race_pred_test[0], 
             di_race_og_all, di_race_pred_all[0]]

gender_list = [di_gender_og_test, di_gender_pred_test[0],
               di_gender_og_all, di_gender_pred_all[0]]


di_df = pd.DataFrame(data={'Race':[round(x, 4) for x in race_list], 
                           'Gender':[round(x, 4) for x in gender_list]}, 
                     index=multi_index)

di_df

In [ ]:
di_melt = di_df.copy()
di_melt['Dataset'] = di_melt.index.map(lambda x: str(x)[1:-1].replace("\'", ""))
di_melt = di_melt.melt(id_vars=['Dataset'], 
                  value_vars=['Race', 'Gender'])

In [ ]:
# plot DI

# Create an array with the colors for plot
colors = ['#142733', '#1d4159', '#23577a', '#4a7d9e', '#231c2e', '#3b2f52', '#594b73', '#85799c']

di_plot = sns.catplot(data=di_melt, 
                          x='variable', y='value', hue='Dataset', palette=sns.color_palette(colors),
                          kind='bar', aspect=2, height=7, legend_out=False)

plt.tick_params(axis='x', which='both',bottom=False, top=False)
di_plot.axes[0][0].set_xlabel('')
di_plot.axes[0][0].set_xticklabels(['Race', 'Gender'], fontsize=16)
di_plot.axes[0][0].set_ylabel(r'$\frac{Pr(MHI=1|non-privileged)}{Pr(MHI=1| privileged)}$', 
                              fontsize=24)

di_plot.axes[0][0].set_axisbelow(True)

plt.axhline(1, color='black', linestyle='dotted', linewidth=2, label='Equal Impact')
plt.axhline(0.8, color='#b8b8b8', linestyle='dotted', linewidth=2, label='80% Impact / Inverse Impact')
plt.axhline(1.25, color='#b8b8b8', linestyle='dotted', linewidth=2)

plt.legend(title='Dataset', fontsize=12, title_fontsize=14)
plt.title('Disparate Impact', fontsize=24)
plt.show()

In [ ]:
def get_stats(X_sub, col):
    
    # get target variable subset
    ind = X_sub.index
    y_sub = y_test[ind]
    
    # for subset, get predictions and probabilities for positive class 
    pred_sub = clf_total.predict(X_sub)
    pred_proba_sub = clf_total.predict_proba(X_sub)
    
    # get FPR, TPR
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_sub, pred_proba_sub[:,1])
    
    # get confusion matrix
    confmat_sub = sklearn.metrics.confusion_matrix(y_sub, pred_sub, labels=[1,0])
    tp = confmat_sub[0,0] # number of true positives
    fp = confmat_sub[1,0] # number of false positives
    fn = confmat_sub[0,1] # number of false negatives
    tn = confmat_sub[1,1] # number of true negatives
    
    # create dataframe with stats
    stats = pd.DataFrame(data={col:[len(pred_sub), # group size
                                    round(sum(pred_sub)/len(pred_sub), 4), #pred prev
                                    round((tp+tn)/len(pred_sub), 4), # accuracy
                                    round(fn/(fn+tp), 4), # fnr
                                    round(fp/(fp+tn), 4), # fpr
                                    round(tp/(tp+fn), 4), # recall
                                    round(sklearn.metrics.auc(fpr, tpr), 4) # auc 
                                   ]}, 
                 index=['Test Set Group Size (N)', 'Predicted Prevalence', 
                        'Accuracy', 'FNR', 'FPR', 'Recall', 'AUC'])
    # transpose df
    stats = stats.T
    stats['Test Set Group Size (N)'] = stats['Test Set Group Size (N)'].astype(int)
    
    # display dataframe with stats
    display(stats)
    
    #plot confusion matrix
    sklearn.metrics.plot_confusion_matrix(clf_total, X_sub, y_sub, labels=[1,0])
    plt.show()
    
    print('\n')
    
    # return dataframe with stats
    return stats
    
    
def subgroup_stats(category1, category2=None):
    '''
    Takes in 1 or 2 pre-OHE column names (e.g., 'race' and 'gender').
    If 2 categories given, produces intersectional statistics.
    
    Note: Does not support intersectional analysis with age
    '''
    # get list of OHE columns for category 1
    if category1 != 'age_at_incident':
        cols_1 = [x for x in X_test.columns if x.startswith(category1+'_')]
        
    # get overall stats
    total_stats = get_stats(X_test, 'Overall')
    
    # intersectional analysis
    if category2 !=None:
        assert category1!='age_at_incident' and category2!='age_at_incident', \
                                         'Intersectional analysis not supported for age'
        
        # get list of OHE columns for category 1
        cols_2 = [x for x in X_test.columns if x.startswith(category2+'_')]
        
        for x in cols_1:
            # for each value of category 1, create subset
            X_sub = X_test[X_test[x]==1]
            
            # continue if subset is empty
            if len(X_sub)==0:
                print(x,'\n\t No members of this subgroup present in the test set. \n')
                continue
                
            # if category 1 subset not empty, create sub-subsets
            else:
            
                for y in cols_2:
                    # create sub-subset for each value of category 2
                    X_sub = X_test[X_test[x]==1]
                    X_sub = X_sub[X_sub[y]==1]
                    if len(X_sub)==0:
                        print(x+' and '+y,
                              '\n\t No members of this subgroup present in the test set. \n')
                    else:
                        # get stats if sub-subset not empty
                        sub_stats = get_stats(X_sub, x+' and '+y)
                        total_stats = pd.concat([total_stats, sub_stats])
                        
    # single trait only
    else:
        if category1 != 'age_at_incident':
            for x in cols_1:
                # for each value of category 1, create subset
                X_sub = X_test[X_test[x]==1]
                if len(X_sub)==0:
                    print(x,'\n\t No members of this subgroup present in the test set. \n')
                else:
                    # get stats if subset not empty
                    sub_stats = get_stats(X_sub, x)
                    total_stats = pd.concat([total_stats, sub_stats])
            
        # special method for age
        else:
            # cut age into bins
            cut = pd.cut(X_test['age_at_incident'], 
                         bins=[X_test['age_at_incident'].min(), 
                               20, 23, 27, 30, 35, 40, 50, 60, 70, 
                               X_test['age_at_incident'].max()+1], 
                         labels=False, right=False, retbins=True)
            
            # save age_at_incident encoded by bin numbers
            age_binned = cut[0]
            
            # save bins
            bins = cut[1]
            
            for x in sorted(age_binned.unique()):
                # create subset for each bin
                this_bin = age_binned[age_binned==x].index
                X_sub = X_test.loc[this_bin]
                
                # create bin labels
                edges = '{}-{}'.format(int(bins[int(x)]), int(bins[int(x)+1]-1))
                if bins[int(x)] == bins[int(x)+1]-1:
                    # if bin contains only 1 age, change label to single number
                    edges = str(int(bins[int(x)]))
            
                if len(X_sub)==0:
                    if not np.isnan(x):
                        print(x,
                              '\n\t No members of this subgroup present in the test set. \n')
                else:
                    # get stats if subset not empty
                    sub_stats = get_stats(X_sub, edges)
                    total_stats = pd.concat([total_stats, sub_stats])
                    
    display(total_stats)
    print('\n')
    
    return total_stats

In [ ]:
def plain_group(x):
    
    split = x.split('_', 1)
    if len(split)==1:
        return split[0].title()
    if len(split)==2:
        return split[1].title()
    return

def plot_stats(stat_df_full, age=False):
    
    stat_df = stat_df_full.iloc[1:]
    
    if not age:
        # Set category (e.g. "Gender")
        cat = stat_df.index[-1].split('_', 1)[0].title()

        # Create column with category values and test set size
        stat_df[cat] = stat_df.index.map(plain_group) + ', N=' + \
                            stat_df['Test Set Group Size (N)'].astype(str)

        stat_df = stat_df.sort_values(by=['Predicted Prevalence'])
        
        # Set palette
        palette='muted'
    
    if age:
        # Set category to "Age"
        cat = 'Age'
        
        # Create column with category values and test set size
        stat_df[cat] = stat_df.index + ', N=' + \
                            stat_df['Test Set Group Size (N)'].astype(str)
        
        # Set sequential palette
        palette = 'GnBu'

    # melt df to prepare for sns.catplot
    df_melt = stat_df.melt(id_vars=[cat], 
                  value_vars=['Predicted Prevalence', 'Accuracy', 
                              'FNR', 'FPR', 'Recall', 'AUC'])

    # plot metrics
    stat_plot = sns.catplot(data=df_melt, 
                              x='variable', y='value', hue=cat,
                              kind='bar', palette=palette,
                            aspect=2, height=6, legend_out=False)
    
    stat_plot.axes[0][0].set_xlabel('')
    stat_plot.axes[0][0].set_ylabel('')
    
    return

In [ ]:
age_stats = subgroup_stats('age_at_incident')
plot_stats(age_stats, age=True)

In [ ]:
gender_stats = subgroup_stats('gender')
plot_stats(gender_stats)

In [ ]:
race_stats = subgroup_stats('race')
plot_stats(race_stats)

In [ ]:
race_and_gender_stats = subgroup_stats('race', 'gender')